# Introduction

This repository uses an implementation of the [Multi Agent Deep Deterministic Policy Gradients (MADDPG)](https://arxiv.org/abs/1706.02275) algorithm. The implementation heavily borrows from the benchmark implementation in the Deep Reinforcement Learning Nanodegree Program. 

The implementation is used to train 2 agents play tennis.  The details of the task can be found in [README.md](README.md) and the code can be executed using the [Collab_Compet_Submission.ipynb](Collab_Compet_Submission.ipynb) Jupyter notebook.


# Multi Agent Deep Deterministic Policy Gradients Algorithm

## Elements

### Actor
* The MADDPG algorithm trains $n_a$ number of actors.
* The state of the actor at time $t$ is denoted by $s_{t,i} \in S_i$ and $i \in \{1,\dots, n_a\}$.
* The actor's actions are denoted by $a_{t,i} \in A_i$. 
* The actor computes actions using the function $\mu_{i}:S_{i} \mapsto A_{i}$.
* In the MADDPG algorithm, the actor is approximated by the function $\mu_{i}:S \times \Theta_{\mu,i} \mapsto A_{i}$ where
   * $\Theta_{\mu,i}$ is set of parameters of the approximation, in this case the weights of the approximating neural network.
* For each actor, the MADDPG algorithm uses 2 neural networks during training hence there are 2 sets of weights:
   * $\theta_{\mu,i} \in \Theta_{\mu,i}$: local neural network weights. These are updated at every training step using policy gradients.
   * $\hat{\theta}_{\mu,i} \in \Theta_{\mu,i}$: target neural network weights. These are updated at a slower rate to improve the stability of training.
* Each actor uses only the local neural network weights when determining actions in actual operation.   

### Critic
* For each actor, the critic approximates the Q function.
* The MADDPG algorithm uses the state of the actor as well as additional signals to estimate the Q function. In this
  implementation each critic uses the states of each actor, the actions of each actor and the state of the ball to estimate the Q function. 
* The use of these additional signals sets the MADDPG algorithm apart from the [DDPG](https://arxiv.org/abs/1509.02971)  algorithm where the critic uses only the state of the actor.
* The observation vector at time $t$ is $o_{t,s}  =\{s_{t,i}\} \in O_{s},\; i \in \{1,\dots, n_a\}$.
* The observation vector at time $t$ is $o_{t,a} =\{a_{t,i}\} \in O_{a},\; i \in \{1,\dots, n_a\}$.
* For each actor, the critic is a function $Q_i:O_{s} \times O_{a} \times \Theta_{q,i} \mapsto \mathbb{R}$ where
  * $\Theta_{q,i}$ is the set of parameters of the neural network used in the approximation.
* For each actor the MADDPG algorithm uses 2 neural networks during training, hence there are 2 sets of parameters:
  * $\theta_{q,i} \in \Theta_q$ is the set of parameters used by the local neural network.
  * $\hat{\theta}_{q,i} \in \Theta_q$ is the set of parameters used by the target neural network and are updated at a lower rate to improve the stability of the training. 
* Actors do not use the critic when determining actions during actual operation.

### Noise Process and Randomizations in Actions
* The MADDPG algorithm uses noise to induce exploration during training. The noise is generated by an [Ornstein - Uhlenbeck process](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process) denoted by $N_{i}(\mu, \theta,\sigma)$ for each actor where 
  * $\mu$ is the mean of the noise
  * $\sigma$ is the volatility
  * $\theta$ is the decay-rate.
* For each actor the MADDPG algorithm computes the action values as follows
  \begin{align*}
   a_{t,i} &= \mu_{i}(s_{t,i}, \theta_{\mu,i}) + \epsilon_t N_i(\mu, \theta, \sigma)\\
   e_{t+1} &= \gamma_\epsilon \epsilon_t 
  \end{align*}
  where the randomness rate $\epsilon_t$, is between [0,1], and $\gamma_\epsilon \in (0,1)$ is the decay rate of $\epsilon_t$ and the initial randomness rate is $\epsilon_0$. 
* Each element of the action vector is clipped so that they are between -1 and 1:
  \begin{align*}
   a_{t,i}[k] \leftarrow \max(\min(1, a_{t,i}[k]), -1)
  \end{align*}

### The Experience Replay Buffer 
* For each actor the MADDPG stores the experience $(s_{t, i}, a_{t,i}, r_{t+1,i}, s_{t+1,i},o_{t,s},o_{t,a},o_{t+1,s})$ where 
  * $s_{t,i}$ is the state at time $t$ of actor $i$,
  * $s_{t+1,i}$ is the state at time $t+1$ of actor $i$, 
  * $a_{t+1,i}$ is the action at time $t$ of actor $i$,
  * $r_{t+1,i}$ is the reward obtained at time $t+1$ of actor i,
  * $o_{t,s}$ is the state observations at time $t$,
  * $o_{t+1,s}$ is the state observations time $t+1$, 
  * $o_{t,a}$ is the action observations at time $t$,
in a buffer of fixed length $N_B$.
* The stored experiences are used in training.
* The experiences are used in batches of size $N_s$.
* The experiences are randomly sampled for each actor independently.

### Critic and Actor Training
* At every $T$ time steps, the local critic neural network is trained as follows:
     1. For $K$ update steps repeat:
        1. For each agent $i \in \{1,\dots, n_a\}$:    
            1. Select a batch of size $N_s$ experiences randomly from the replay buffer.
            2. For each experience , compute:
               \begin{align*}
               \hat{o}_{t+1,a,j} & = \{\mu_{k}(s_{t+1,k,j}, \theta_{\mu,k}),\; k \in 1,\dots, n_a\}\;\; \forall j \in \{1, \dots, N_s\} \\
               y_{i,j} &= r_{t+1,i,j} + \gamma Q(o_{t+1,s,j}, \hat{o}_{t+1,a,j},\hat{\theta}_{q,i})\;\; \forall j \in \{1, \dots, N_s\}
               \end{align*}
               where $\gamma$ is the discount factor.
            3. Update $\theta_{q,i}$ as follows:
               \begin{align*}
               \theta_{q,i} &\leftarrow \theta_{q,i} + \frac{\alpha_q}{N_s} \sum_{j=1}^{N_s}(y_{i,j}- Q(o_{t,s,j},o_{t,a,j}, \theta_{q,i}))\nabla_{\theta_{q,i}} Q(o_{t,s,j},o_{t,a,j}, \theta_{q,i})
               \end{align*}
               to minimize the objective function
               \begin{align*}
                L = \frac{1}{N_s}\sum_{j =1}^{N_s} (y_{i,j}- Q(o_{t,s,j},o_{t,a,j}, \theta_{q,i}))^2 
               \end{align*} 
               where $\alpha_q$ is the critic learning rate.       

* For each actor,  the local actor neural network weights are updated using a policy gradient approach.
* At every $T$ time steps, the local critic neural network is trained as follows:
     1. For $K$ update steps repeat:
        1. For each agent $i \in \{1,\dots, n_a\}$:
            1. Select a batch of size $N_s$ experiences randomly from the replay buffer.
            2. Compute the policy gradient and update the weights.
            \begin{align*}
                \nabla J_{\theta_{\mu,i}} &= \frac{1}{N_s}\sum_{j=1}^{N_s}\nabla_{a_{i}} Q_{i}(o_{t,s,j},o_{t,a,j}, \hat{\theta}_{q,i})\nabla_{\theta_{\mu,i}}\mu(s_{t,i,j}, \theta_{\mu,i}) \\
            \theta_{\mu,i} &\leftarrow \theta_{\mu,i} + \alpha_\mu \nabla J_{\theta_{\mu,i}}
            \end{align*}
            where $\alpha_\mu$ is the learning rate for the actor. 

### Continuous Target Actor and Critic Update
* For each agent, the target actor and critic neural network weights are updated gradually at every $T$ time steps and every time the local neural networks are as follows: 
 \begin{align*}
   \hat{\theta}_{q,i} &\leftarrow  (1-\tau)\hat{\theta}_{q,i} + \tau\theta_{q,i} \\
   \hat{\theta}_{\mu,i} &\leftarrow  (1-\tau)\hat{\theta}_{\mu,i} + \tau\theta_{\mu,i}
  \end{align*}
where $\tau$ is the soft update parameter.

# Overall Algorithm
0. For each agent $i \in 1,\dots, n_a$
   * Initialize $\theta_{\mu,i}$ and $\theta_{q,i}$ randomly. 
   * Set $\hat{\theta}_{\mu,i} = \theta_{\mu,i}$ and $\hat{\theta}_{q,i} = \theta_{q,i}$. 
   * Set the parameters $\tau$,$\gamma$, $\alpha_\mu$, $\alpha_q$, $N_B$, $N_s$, $\mu$, $\theta$, $\sigma$, $\epsilon_0$, $\gamma_\epsilon$, $T$, $K$, $M$ where $M$ is the maximum number of episodes.  
   
1. for $M$ episodes do:
   1. For each actor initialize random process $N_{i}(\mu,\theta,\sigma)$.
   2. Receive initial states $\{s_{1,i},\; i \in 1, \dots, n_a\}$ and initial state observation $\{o_{1,s}\}$.
   3. Until episode is over do:
      1. For each actor $i$, compute $\{a_{t,i}\}$ as  
          \begin{align*}
            a_{t,i} &= \mu(s_{t,i}, \theta_{\mu,i}) + \epsilon_t N_{i}(\mu, \theta, \sigma)\\
            a_{t,i}[k] &\leftarrow \max(\min(1, a_{t,i}[k]), -1)\\
            e_{t+1}  &= \gamma_\epsilon \epsilon_t 
          \end{align*}
      2. For each actor, execute $a_{t,i}$, receive $r_{t+1,i}$ and transition to new state $s_{t+1,i}$.
      3. For each actor, store experience $\{s_{t,i}, a_{t,i}, r_{t+1,i}, s_{t+1,i}, o_{t,s}, o_{t,a}, o_{t+1,s}\}$ in the experience replay buffer.
      4. If $t\bmod T = 0$ and the number of experiences stored in the replay buffer is greater than $N_s$ then for each agent do:
        1. for $u = 1,\; K$ do
            1. Obtain a batch of experiences from the replay buffer of size $N_s$.
            2. Update the critic local network weights. 
                 * For each experience , compute:
              \begin{align*}
                  \hat{o}_{t+1,a,j} & = \{\mu_{k}(s_{t+1,k,j}, \theta_{\mu,k}),\; k \in 1,\dots, n_a\}\;\; \forall j \in \{1, \dots, N_s\} \\
               y_{i,j} &= r_{t+1,i,j} + \gamma Q(o_{t+1,s,j}, \hat{o}_{t+1,a,j},\hat{\theta}_{q,i})\;\; \forall j \in \{1, \dots, N_s\}
               \end{align*}
            3. Update $\theta_{q,i}$ as follows:
               \begin{align*}
               \theta_{q,i} &\leftarrow \theta_{q,i} + \frac{\alpha_q}{N_s} \sum_{j=1}^{N_s}(y_{i,j}- Q(o_{t,s,j},o_{t,a,j}, \theta_{q,i}))\nabla_{\theta_{q,i}} Q(o_{t,s,j},o_{t,a,j}, \theta_{q,i})
               \end{align*}
                       
            4. Update the actor local network weights using the policy gradient:   
               \begin{align*}
                \nabla J_{\theta_{\mu,i}} &= \frac{1}{N_s}\sum_{j=1}^{N_s}\nabla_{a_{i}} Q_{i}(o_{t,s,j},o_{t,a,j}, \hat{\theta}_{q,i})\nabla_{\theta_{\mu,i}}\mu(s_{t,i,j}, \theta_{\mu,i}) \\
               \theta_{\mu,i} &\leftarrow \theta_{\mu,i} + \alpha_\mu \nabla J_{\theta_{\mu,i}}
               \end{align*}
            5. Soft update the critic target and actor target neural network weights.   
                \begin{align*}
                   \hat{\theta}_{q,i} &\leftarrow  (1-\tau)\hat{\theta}_{q,i} + \tau\theta_{q,i} \\
                   \hat{\theta}_{\mu,i} &\leftarrow  (1-\tau)\hat{\theta}_{\mu,i} + \tau\theta_{\mu,i}
                \end{align*} 


# Implementation Details
## Neural Network Structure

### Critic Training
* When training the local critic neural networks, the [Huber loss](https://en.wikipedia.org/wiki/Huber_loss)
  is minimized instead of the mean squared error. The Huber loss is less sensitive to outliers. The desired performance criterion was attained with fewer episodes during training when the Huber loss was used.

### Actor Target and Local Neural Networks

* The structure is as follows for both the actor local and actor target neural networks:
    1. Layer: Input Layer(input_dim = size of the state vector for each actor, output_dim = 512)
    2. Normalization: Batch normalization(input_dim = 512, output_dim = 512)
    3. Activation: Relu Activation
    4. Layer: Hidden Layer(input_dim = 512, output_dim = 512)
    5. Activation: Relu Activation
    6. Layer: Hidden Layer(input_dim = 512, output_dim = 256)
    7. Normalization: Batch normalization(input_dim = 512, output_dim = 256) 
    8. Layer: Output Layer(input_dim = 256, output_dim = action vector size (2))
    9. Activation: Tanh
    
### Critic Target and Local Neural Networks

* The structure is as follows for both the 
    1.  Layer: Input Layer (input_dim = size of the state observation vector,output_dim = 512)
    2.  Normalization: Batch Normalization
    3.  Activation: Relu
    4.  Layer: Hidden (input_dim = 512 + size of the action observation vector, output_dim = 512)
    5.  Activation: Relu
    6.  Layer: Hidden Layer(input_dim = 512, output_dim = 512)
    7.  Activation: Relu
    8.  Layer: Hidden Layer(input_dim  = 512, output_dim = 256)
    9.  Activation: Relu
    10. Layer: Output Layer(input_dim  = 256, output_dim = 1) 
* Using a nework with more layers and more units per layer helped attain the desired performance criterion in fewer number of episodes. 

## Hyperparameters

* Soft update parameter,$\tau$: 0.001
* Discount factor, $\gamma$:0.99
* Learning rate for the actor local training, $\alpha_\mu$:0.0001
* Learning rate for the critic local training, $\alpha_q$:0.0001
* Replay buffer size, $N_B$: 1,000,000
* Training batch size $N_s$: 256
* Ornstein-Uhlenbeck mean, $\mu$: 0.0
* Ornstein-Uhlenbeck decay rate, $\theta$:0.15
* Ornstein-Uhlenbeck volatility, $\sigma$:0.2
* Initial randomness rate, $\epsilon_0$: 1.0
* Randomness rate decay rate, $\gamma_\epsilon$: 0.999
* Frequency of update for the neural networks, $T$: 1
* Number of steps taken at every update, $K$: 5



# Training Results
 The agents achieve a moving average maximum score  of 0.5 or above in about 705 episodes. 
![Training Performance](training_performance.png)


# Future Directions
* The hyper parameters for the algorithm have been chosen by trial and error. Better parameters can be found by using automated hyperparameter search techniques.
* The performance of the agents from episode to episode varies a lot. The big question is how to ensure consistent performance between episodes. I tried increasing the neural network sizes, adding batch normalization and using Huber loss instead of mean squared error loss. All these helped with training times and achieving the desired training performance, but reduced the variability only slightly.
* I observed that successful episodes are frequently followed by episodes where the performance is very unsatisfactory. I think this is because the experiences from low performance episodes might be dropped from the replay buffers. I think experience where the agents failed should be used more often in training. As a next step, I would like to implement [Prioritized experienced replay](https://arxiv.org/abs/1511.05952) where failures have higher priorities.
* Finally I would like to experiment with  other algorithms in the context of multi agent reinforcement learning such as 
   * [Trust Region Policy Optimization](https://arxiv.org/abs/1502.05477)
   * [D4PG](https://openreview.net/pdf?id=SyZipzbCb):  This algorithm includes prioritized experience replay and is shown to achieve better performance quicker than DDPG. 